In [1]:
from document_ai import DocumentProcessor
from document_ai.parser import DigitalPDFParser
from document_ai.formatter import PDFFormatter
from document_ai.schemas import Document

In [2]:
document = Document(
    document_type="digital_pdf",
    uri="/Users/zeel/Downloads/MCP tool review.pdf"
)

In [3]:
processed_document = DocumentProcessor(
    parser=DigitalPDFParser(),
    formatter=PDFFormatter()
).process(document)

pages=[Page(lines=[Line(text='MMCCPP SSeerrvveerr RReevviieeww:: RRoolleess aanndd RReessppoonnssiibbiilliittiieess AAnnaallyyssiiss', bounding_box=BoundingBox(x0=0.0552751668613708, top=0.03729497879854281, x1=0.7538760996328405, bottom=0.057253040379302955)), Line(text="WWhhaatt''ss WWoorrkkiinngg WWeellll \x00\x00", bounding_box=BoundingBox(x0=0.05544853455191045, top=0.07955689812102706, x1=0.2622188204208475, bottom=0.09452544430659707)), Line(text='11.. CClleeaarr SSeeppaarraattiioonn ooff CCoonncceerrnnss', bounding_box=BoundingBox(x0=0.05546429525105043, top=0.11931570676255383, x1=0.2827223552316084, bottom=0.13072031338013101)), Line(text='Vehicle search is separate from business info search', bounding_box=BoundingBox(x0=0.09631602742184874, top=0.14499277761882737, x1=0.37996649935317955, bottom=0.15354623258201033)), Line(text='Contact management is distinct from booking operations', bounding_box=BoundingBox(x0=0.09631602742184874, top=0.15568459632280596, x1=0.402943833500

In [4]:
processed_document.model_dump()

{'document_type': 'digital_pdf',
 'uri': '/Users/zeel/Downloads/MCP tool review.pdf',
 'content': {'pages': [{'lines': [{'text': 'MMCCPP SSeerrvveerr RReevviieeww:: RRoolleess aanndd RReessppoonnssiibbiilliittiieess AAnnaallyyssiiss',
      'bounding_box': {'x0': 0.0552751668613708,
       'top': 0.03729497879854281,
       'x1': 0.7538760996328405,
       'bottom': 0.057253040379302955}},
     {'text': "WWhhaatt''ss WWoorrkkiinngg WWeellll \x00\x00",
      'bounding_box': {'x0': 0.05544853455191045,
       'top': 0.07955689812102706,
       'x1': 0.2622188204208475,
       'bottom': 0.09452544430659707}},
     {'text': '11.. CClleeaarr SSeeppaarraattiioonn ooff CCoonncceerrnnss',
      'bounding_box': {'x0': 0.05546429525105043,
       'top': 0.11931570676255383,
       'x1': 0.2827223552316084,
       'bottom': 0.13072031338013101}},
     {'text': 'Vehicle search is separate from business info search',
      'bounding_box': {'x0': 0.09631602742184874,
       'top': 0.1449927776188273